 ## <b>Connecting vxi11 compatible devices via Ethernet </b>

In [10]:
!pip3 install python-vxi11

In [5]:
import vxi11
import time

In [6]:
keysight_ip = '192.168.0.8'

In [7]:
Keysight = vxi11.Instrument(keysight_ip)
print(Keysight.ask("*IDN?"))

KeyboardInterrupt: 

In [210]:
rigol_ip = '192.168.0.14'

In [211]:
Rigol = vxi11.Instrument(rigol_ip)
print(Rigol.ask("*IDN?"))

RIGOL TECHNOLOGIES,DS1074Z,DS1ZA193816619,00.04.04.SP3


## <b> Interacting with a Brüel & Kjær Type 3050-B-6 using Python </b>

In [212]:
cd ../PyBnK

C:\Users\ricky\Documents\VM3000-Microphones\PyBnK


In [214]:
from bnk.bnk import WavHeader, OpenWav, Instrument

In [216]:
bnk_ip = '192.168.0.9'
ADAC = Instrument(bnk_ip)
print(ADAC) # Show some info about the BnK device


	BnK state : RecorderOpened
	commands sent : 6
	BnK clock : Fri, 26 Feb 2021 05:48:10 GMT

BnK Properties:
    6 channels
    SD card is inserted
    Filters      : ['DC', '0.1 Hz 10%', '0.7 Hz', '1.0 Hz 10%', '7.0 Hz', '22.4 Hz', 'Intensity']
    SampleRates  : [131072, 65536, 32768, 16384, 8192, 4096]        
    Ranges       : ['10 Vpeak', '31.6 Vpeak']

	Default setup
	Channel 1 : Channel 1
		131072 SPS, 7.0 Hz filter, 10 Vpeak, 1V/V.
	Channel 2 : Channel 2
		131072 SPS, 7.0 Hz filter, 10 Vpeak, 1V/V.
	Channel 3 : Channel 3
		131072 SPS, 7.0 Hz filter, 10 Vpeak, 1V/V.
	Channel 4 : Channel 4
		131072 SPS, 7.0 Hz filter, 10 Vpeak, 1V/V.
	Channel 5 : Channel 5
		131072 SPS, 7.0 Hz filter, 10 Vpeak, 1V/V.
	Channel 6 : Channel 6
		131072 SPS, 7.0 Hz filter, 10 Vpeak, 1V/V.

	BnK state : RecorderOpened
	commands sent : 6
	BnK clock : Fri, 26 Feb 2021 05:48:11 GMT


## <b> Python Wrapper for Keysight 33500B - Modified from Carlin's code</b>

In [3]:
class keysight():
    
    def __init__(self,instrument_ID=keysight_ip):
        self.Keysight =  vxi11.Instrument(instrument_ID)
        print(self.Keysight.ask("*IDN?"))
        
    def enable(self):
        self.Keysight.write("OUTPut ON")
        
    def disable(self):
        self.Keysight.write("OUTPut OFF") 
        
    def enable_high_impedance(self):
        self.Keysight.write("OUTPut:LOAD INF")  
        
    def sine(self,frequency=500,amplitude=0.01,offset=0):
        self.Keysight.write(f"VOLTage:OFFSet {offset}")
        self.Keysight.write("VOLTage:UNIT VRMS")
        self.Keysight.write(f"VOLTage {amplitude}")
        self.Keysight.write("FUNCtion SINusoid")
        self.Keysight.write(f"FREQuency {frequency}")
        
    def whitenoise(self, bandwidth=1500, amplitude=0.01):
        self.Keysight.write("VOLTage:UNIT VRMS")
        self.Keysight.write(f"VOLTage {amplitude}")
        self.Keysight.write("FUNCtion NOISe")
        self.Keysight.write(f"FUNCtion:NOISe:BANDwidth {bandwidth}")
        
    def enable_burst(self,frequency=2000,ncycles=50,int_period=5):
        self.Keysight.write("FUNCtion SINusoid")
        self.Keysight.write("BURSt:STATe ON")
        self.Keysight.write(f"FREQuency {frequency}")
        self.Keysight.write(f"BURSt:NCYCles {ncycles}")
        self.Keysight.write(f"BURSt:INTernal:PERiod {int_period}")          
        self.Keysight.write("OUTPut ON")
        
    def disable_burst(self):
        self.Keysight.write("BURSt:STATe OFF")
        self.Keysight.write("OUTPut OFF")
        
    def enable_sweep(self, mode="logarithmic", sweep_period=10):
        self.Keysight.write("FUNCtion SINusoid")
        self.Keysight.write("SWEep:STATe ON")
        self.Keysight.write(f"SWEep:SPAcing {mode}")
        self.Keysight.write(f"SWEep:TIME {sweep_period}")
        self.Keysight.write("OUTPut ON")
        
    def disable_sweep(self):
        self.Keysight.write("SWEep:STATe OFF")
        self.Keysight.write("OUTPut OFF")

## Connecting to RPI Zero W

In [ ]:
#Connecting to the RPi over Wi-Fi
from paramiko import SSHClient
from scp import SCPClient
RPi = '192.168.0.6' # This is the IP address when RPi is connected to SHL network, note that this changes depending on different networks unless static IP is set
ssh = SSHClient()
ssh.load_system_host_keys()
ssh.connect(RPi,username='pi')

In [ ]:
scp = SCPClient(ssh.get_transport())

In [ ]:
stdin, stdout, stderr = ssh.exec_command('ls')
print(stdout)

## Test Cases

In [4]:
sg=keysight()

sg.enable_high_impedance()

NameError: name 'vxi11' is not defined

In [ ]:
import paramiko
import time
import datetime

In [ ]:
# Sweep Test

mode = "logarithmic"
sweep_period = 5

sg.enable_sweep(mode, sweep_period)
sg.enable()
time.sleep(sweep_period)
sg.disable()
time.sleep(sweep_period)
sg.disable_sweep()

print("Finished Signal Output")

In [375]:
# Continuous Sine Wave Test

frequency = 2000
amplitude = 0.01 # Vrms
offset = 0
sleep_time = 2

sg.sine(frequency,amplitude,offset)
sg.enable()
time.sleep(sleep_time)
sg.disable()
time.sleep(sleep_time)

print("Finished Signal Output")

Finished


In [376]:
# White Noise Test

bandwidth = 500
amplitude = 0.01 # Vrms
sleep_time = 2

sg.whitenoise(bandwidth,amplitude)
sg.enable()
time.sleep(sleep_time)
sg.disable()
time.sleep(sleep_time)

print("Finished Signal Output")

Finished


In [377]:
# Burst Sweep Test

frequency = 2000
ncycles = 10
int_period = 1
sleep_time = 2

sg.enable_burst(frequency,ncycles,int_period)
  
for f in range(2000,5000,1000):
    print(f'Pulsing at {f} Hz')
    Keysight.write(f"FREQuency {f}")
    sg.enable()
    time.sleep(sleep_time)
    sg.disable()
    time.sleep(sleep_time)

sg.disable_burst()
    
print("Finished Signal Output")

Pulsing at 2000 Hz
Pulsing at 3000 Hz
Pulsing at 4000 Hz
Finished


In [378]:
# Burst Test Constant Frequency

frequency = 2000
ncycles = 10
int_period = 1
sleep_time = 2

sg.enable_burst(frequency,ncycles,int_period)

for i in range(1,11,1):
    print(f'Pulse number {i}')
    Keysight.write(f"FREQuency {frequency}")
    sg.enable()
    time.sleep(sleep_time)
    sg.disable()
    time.sleep(sleep_time)
          
print("Finished Signal Output")

Pulse number 1
Pulse number 2
Pulse number 3
Pulse number 4
Pulse number 5
Pulse number 6
Pulse number 7
Pulse number 8
Pulse number 9
Pulse number 10
Finished


In [ ]:
now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
sample_rate = 48000

cmd = f'arecord -D plughw:1 -c2 -r {sample_rate} -f S32_LE -t wav -V stereo -v -d {sleep_time} test_{now}.wav'
print(cmd)
ssh.exec_command(cmd)

cli = paramiko.client.SSHClient()
cli.set_missing_host_key_policy(paramiko.client.AutoAddPolicy())
cli.connect(hostname=RPi, username="pi")
stdin_, stdout_, stderr_ = cli.exec_command(cmd)
stdout_.channel.recv_exit_status()
lines = stdout_.readlines()
for line in lines:
    print(line)

cli.close()
scp.get(f'test_{now}.wav')